In [42]:
import openai, json, requests

client = openai.OpenAI()

# ============================================================
# 1. Memory structures
# ============================================================
messages = []          # Full conversation history (LLM memory)
user_profile = {       # Structured memory for quick reference
    "favorite_genres": [],
    "watched_movies": [],
}

In [43]:
# ============================================================
# 2. System prompt — instructs the LLM HOW to use memory
# ============================================================
SYSTEM_PROMPT = """You are a Movie Expert Chatbot that remembers everything the user tells you.

CAPABILITIES:
- Look up currently popular movies
- Get detailed information about any movie by ID
- Find similar movies to any given movie by ID

BEHAVIOR:
- When the user asks about popular/trending movies, call get_popular_movies.
- When the user asks about a specific movie's details, call get_movie_details.
- When the user asks for movies similar to one, call get_similar_movies.
- You may chain multiple tool calls if needed (e.g., get details then find similar).
- Always base your answers on real API data, not your training knowledge.

MEMORY RULES:
- Remember the user's favorite genres across the conversation.
- Remember movies the user has already watched.
- NEVER recommend a movie the user said they already watched.
- Reference what you know about the user to personalize responses.

Be enthusiastic about movies."""

messages.append({"role": "system", "content": SYSTEM_PROMPT})


In [44]:
# ============================================================
# 3. Tool functions — real API calls
# ============================================================
BASE_URL = "https://nomad-movies.nomadcoders.workers.dev"


def get_popular_movies():
    """Fetch popular movies from /movies."""
    response = requests.get(f"{BASE_URL}/movies")
    response.raise_for_status()
    return json.dumps(response.json(), ensure_ascii=False)


def get_movie_details(id):
    """Fetch movie details from /movies/:id."""
    response = requests.get(f"{BASE_URL}/movies/{id}")
    response.raise_for_status()
    return json.dumps(response.json(), ensure_ascii=False)


def get_similar_movies(id):
    """Fetch similar movies from /movies/:id/similar."""
    response = requests.get(f"{BASE_URL}/movies/{id}/similar")
    response.raise_for_status()
    return json.dumps(response.json(), ensure_ascii=False)


FUNCTION_MAP = {
    "get_popular_movies": get_popular_movies,
    "get_movie_details": get_movie_details,
    "get_similar_movies": get_similar_movies,
}

In [45]:
# ============================================================
# 4. Tool schemas — parameter
# ============================================================
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_popular_movies",
            "description": "Get a list of currently popular movies. Call this when the user asks about trending, popular, or what's playing now.",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": [],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_movie_details",
            "description": "Get detailed information about a specific movie by its ID, including title, overview, release date, rating, genres, etc.",
            "parameters": {
                "type": "object",
                "properties": {
                    "id": {
                        "type": "string",
                        "description": "The numeric ID of the movie.",
                    },
                },
                "required": ["id"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_similar_movies",
            "description": "Get a list of movies similar to a specific movie by its ID. Call this when the user asks for recommendations based on a specific movie.",
            "parameters": {
                "type": "object",
                "properties": {
                    "id": {
                        "type": "string",
                        "description": "The numeric ID of the movie to find similar movies for.",
                    },
                },
                "required": ["id"],
            },
        },
    },
]

In [46]:
# ============================================================
# 5. Memory Helper
# ============================================================
def update_user_profile(user_msg: str):
    """Side LLM call to extract genres & watched movies into structured memory."""
    extraction_response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": (
                    "Extract any favorite genres and already-watched movie titles "
                    "from the user message. Return ONLY valid JSON: "
                    '{"genres": [...], "watched": [...]}. '
                    "Empty lists if nothing found. No markdown."
                ),
            },
            {"role": "user", "content": user_msg},
        ],
    )
    try:
        extracted = json.loads(extraction_response.choices[0].message.content)
        for genre in extracted.get("genres", []):
            if genre not in user_profile["favorite_genres"]:
                user_profile["favorite_genres"].append(genre)
        for movie in extracted.get("watched", []):
            if movie not in user_profile["watched_movies"]:
                user_profile["watched_movies"].append(movie)
    except (json.JSONDecodeError, AttributeError):
        pass


def build_profile_reminder():
    """Returns a memory snapshot string or None."""
    if not user_profile["favorite_genres"] and not user_profile["watched_movies"]:
        return None
    parts = []
    if user_profile["favorite_genres"]:
        parts.append(f"Favorite genres: {', '.join(user_profile['favorite_genres'])}")
    if user_profile["watched_movies"]:
        parts.append(f"Already watched (DO NOT recommend these): {', '.join(user_profile['watched_movies'])}")
    return "[MEMORY SNAPSHOT]\n" + "\n".join(parts)

In [47]:
# ============================================================
# 6. Main loop - Engine
# ============================================================
from openai.types.chat import ChatCompletionMessage


def process_ai_response(message: ChatCompletionMessage):
    """
    Handles the LLM's response. Two branches:
      A) tool_calls present → execute tools, record results, LOOP BACK to call_ai()
      B) no tool_calls     → final text answer, print and stop
    """
    # --- Branch A: LLM wants to call tool(s) ---
    if message.tool_calls:
        # Record the assistant's tool-call request in memory
        messages.append(
            {
                "role": "assistant",
                "content": message.content or "",
                "tool_calls": [
                    {
                        "id": tc.id,
                        "type": "function",
                        "function": {
                            "name": tc.function.name,
                            "arguments": tc.function.arguments,
                        },
                    }
                    for tc in message.tool_calls
                ],
            }
        )

        # Execute each requested tool
        for tool_call in message.tool_calls:
            fn_name = tool_call.function.name
            raw_args = tool_call.function.arguments

            print(f"  [Tool Call] {fn_name}({raw_args})")

            try:
                parsed_args = json.loads(raw_args)
            except json.JSONDecodeError:
                parsed_args = {}

            fn = FUNCTION_MAP.get(fn_name)
            if fn is None:
                result = json.dumps({"error": f"Unknown function: {fn_name}"})
            else:
                try:
                    result = fn(**parsed_args)
                except Exception as e:
                    result = json.dumps({"error": str(e)})

            print(f"  [Tool Result] {result[:200]}{'...' if len(result) > 200 else ''}")

            # Record tool result in memory
            messages.append(
                {
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "name": fn_name,
                    "content": result,
                }
            )

        # >>> LOOP BACK — let the LLM read tool results and decide next step
        call_ai()

    # --- Branch B: Final text answer (no more tool calls) ---
    else:
        messages.append({"role": "assistant", "content": message.content})
        print(f"\nAI: {message.content}")


def call_ai():
    """Single LLM call with memory snapshot injected."""
    # Build payload: full history + fresh profile reminder
    call_messages = list(messages)
    reminder = build_profile_reminder()
    if reminder:
        call_messages.append({"role": "system", "content": reminder})

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=call_messages,
        tools=TOOLS,
    )
    process_ai_response(response.choices[0].message)

In [48]:
# ============================================================
# 7. Chat
# ============================================================
def chat(user_input: str):
    """Main entry: update profile, append message, kick off agentic loop."""
    update_user_profile(user_input)
    messages.append({"role": "user", "content": user_input})
    print(f"\nUser: {user_input}")
    call_ai()

In [ ]:
# ============================================================
# 8. Main loop
# ============================================================
if __name__ == "__main__":
    print("=" * 50)
    print("  Movie Expert Agent (Complete Agentic Loop)")
    print("=" * 50)
    print('Type your question (or "q" to quit)\n')

    while True:
        user_input = input("You: ")
        if user_input.lower() in ("quit", "q"):
            break
        chat(user_input)
        print()

    print("\n--- Final Memory ---")
    print(json.dumps(user_profile, ensure_ascii=False, indent=2))


  Movie Expert Agent (Complete Agentic Loop)
Type your question (or "q" to quit)


User: Tell me mostfavorite movies right now 
  [Tool Call] get_popular_movies({})
  [Tool Result] [{"adult": false, "backdrop_path": "https://image.tmdb.org/t/p/w1280/6YjnTRBz704LF1uJ3ZC4wsS9T8r.jpg", "genre_ids": [28, 80, 53], "id": 1290821, "original_language": "en", "original_title": "Shelter",...

AI: Here are some of the most popular movies right now:

1. **Shelter**
   - **Release Date:** January 28, 2026
   - **Overview:** A man living in self-imposed exile on a remote island rescues a young girl from a violent storm, leading him to confront enemies tied to his past.
   - **Rating:** 6.94
   - **[Poster](https://image.tmdb.org/t/p/w780/buPFnHZ3xQy6vZEHxbHgL1Pc6CR.jpg)**

2. **Mercy**
   - **Release Date:** January 20, 2026
   - **Overview:** In the near future, a detective is on trial for murdering his wife and has ninety minutes to prove his innocence to an AI judge.
   - **Rating:** 7.125
   - *